# Imports

In [1]:
import pynq
import time
from pynq import PL
import xrfclk
import xrfdc
import numpy as np
from fpbinary import FpBinary
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq, fftshift
import mkidgen3 as g3
import mkidgen3.mkidpynq as mkidpynq
import mkidgen3.daccomb as daccomb
import mkidgen3
from mkidgen3.daccomb import generateTones
from mkidgen3.drivers import axiswitch, bintores, capture, ddc, dactable, axififo, rfdc

# Load the Overlay and Mig Allocation XCLBIN

In [2]:
ol = pynq.Overlay('filter_2cap/gen3_512_iqsweep.bit', ignore_version=True, download=True)
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")
#xrt_ol = pynq.Overlay('mig_modified_ip_layout_mem_topology.xclbin', device=pynq.Device.devices[1], download=True)

PL Bitfile: /home/xilinx/jupyter_notebooks/Unit_Tests/Full_Channelizer/filter_2cap/gen3_512_iqsweep.bit
PL Timestamp: 2022/1/10 20:16:28 +521528
Overlay timestamp: 2022/1/10 20:16:28 +521528  Loaded: True


Tell mkidgen3

In [3]:
mkidpynq.PL_DDR4_ADDR = 0x500000000
g3._gen3_overlay=ol
# g3._mig_overlay=xrt_ol

## Fire Up the RFDC

In [ ]:
#xrfclk.set_all_ref_clks(409.6)
# new driver: xrfclk.set_ref_clks(409.6)

In [4]:
rfdc=ol.usp_rf_data_converter_0
adc_tile = rfdc.adc_tiles[2]
adc0 = adc_tile.blocks[0]
adc1 = adc_tile.blocks[1]
dac_tile = rfdc.dac_tiles[0]
dac0 = dac_tile.blocks[0]
dac1 = dac_tile.blocks[1]

In [5]:
dac0.QMCSettings = {
    'EnableGain': 1,
    'EnablePhase': 0,
    'EventSource': 0,
    'GainCorrectionFactor': 1.9,
    'OffsetCorrectionFactor': 0,
    'PhaseCorrectionFactor': 0.0
}
dac0.UpdateEvent(xrfdc.EVENT_QMC)
dac1.QMCSettings = {
    'EnableGain': 1,
    'EnablePhase': 0,
    'EventSource': 0,
    'GainCorrectionFactor': 1.9,
    'OffsetCorrectionFactor': 0,
    'PhaseCorrectionFactor': 0.0
}
dac0.UpdateEvent(xrfdc.EVENT_QMC)
mkidgen3.drivers.rfdc.status()

10
ADC0
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common Status:  71
  Tile Disable:  0
ADC1
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common Status:  71
  Tile Disable:  0
ADC2
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common Status:  12015
  Tile Disable:  0
ADC3
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common Status:  71
  Tile Disable:  0
DAC0
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common Status:  54351
  Tile Disable:  0
DAC1
  Restart Power-On State Machine:  0
  Restart State:  15
  Current State:  15
  Reset Count:  0
  Interrupt Status:  0
  Tile Common St

## Fetch Blocks

In [7]:
rfdc=ol.usp_rf_data_converter_0
cap=ol.capture
#switch = ol.capture.axis_switch_0
#fifo=ol.capture.axi_fifo_mm_s_0
replay = ol.dac_table_axim_0
axiint = ol.axi_intc_0
ddc=ol.photon_pipe.reschan.resonator_ddc
bin2res = ol.photon_pipe.reschan.bin_to_res

In [ ]:
self=fifo
addr = self.register_map.RDFD.address
occ = self.rx_occupancy
data = []
for _ in range(occ):
    x=self.mmio.read(addr)
    data.extend([x & 0xffff, x>>16] )
    

print(occ, data[:10])

In [ ]:
switch.set_master(1,commit=True)

### Allocate a Chunk of PLDRAM

In [ ]:
pl_buf = pynq.allocate((2**29,2), 'i2', target=xrt_ol.MIG0) # 2**30 is 4 GiB

In [ ]:
pl_buf[:]

In [10]:
list(range(0,128,2))

[0,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 38,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 98,
 100,
 102,
 104,
 106,
 108,
 110,
 112,
 114,
 116,
 118,
 120,
 122,
 124,
 126]

In [9]:
cap.filter_iq.keep=range(0,128,2)
cap.filter_iq.register_map

RegisterMap {
  keep_1 = Register(keep=1431655765),
  keep_2 = Register(keep=1431655765),
  keep_3 = Register(keep=1431655765),
  keep_4 = Register(keep=1431655765),
  keep_5 = Register(keep=0),
  keep_6 = Register(keep=0),
  keep_7 = Register(keep=0),
  keep_8 = Register(keep=0),
  lastgrp = Register(lastgrp=126, RESERVED=0)
}

In [ ]:
cap.write_axi256_0.capture(128, pl_buf.device_address)

In [11]:
cap.write_axi256_0.capture(128, 0x500000000)

In [13]:
cap.write_axi256_0.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, RESERVED_3=0, RESERVED_4=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  capturesize = Register(capturesize=128, RESERVED=0),
  out_r_1 = Register(out_r=0),
  out_r_2 = Register(out_r=5)
}

## Setup DAC REPLAY

In [ ]:
f0=6.617570
span=0.002
offset=.5

In [ ]:
g3._gen3_overlay.dac_table_axim_0.register_map

In [ ]:
freq = np.array([(offset+0)*1e9])
g3.set_waveform(freq)
g3._gen3_overlay.dac_table_axim_0.register_map

In [ ]:
g3._gen3_overlay.dac_table_axim_0._buffer[:10]

In [ ]:
g3.set_channels(freq)
g3._gen3_overlay.photon_pipe.reschan.bin_to_res.bins[:20]

In [ ]:
freq = np.array([(offset+0)*1e9, .421*1e9])
g3.set_tones(freq)
ol.photon_pipe.reschan.resonator_ddc.tones

In [ ]:
g3._gen3_overlay.capture.axis_switch_0.set_master(1, commit=True)  # After the FIR

In [ ]:
n=100

In [ ]:
#4 bytes per IQ * n * np.ceil(_frequencies.size/8)
buf = pynq.allocate((2*n*int(np.ceil(freq.size/8)), 2), 'i2', target=g3._mig_overlay.MIG0)  # 2**30 is 4 GiB
buf[:]=0

In [ ]:
grp= np.arange(np.ceil(freq.size/8), dtype=int)
n, grp

In [ ]:
g3._gen3_overlay.capture.iq_capture_0.register_map

In [ ]:
captime=g3._gen3_overlay.capture.iq_capture_0.capture(n, groups=grp, device_addr=0, start=False)
g3._gen3_overlay.capture.iq_capture_0.start()
g3._gen3_overlay.capture.iq_capture_0.register_map,time.sleep(captime*2),g3._gen3_overlay.capture.iq_capture_0.register_map

In [ ]:
g3._gen3_overlay.capture.iq_capture_0.start()

In [ ]:
g3._gen3_overlay.capture.iq_capture_0.register_map

In [ ]:
#     #TODO we need to wait for the core to signal done
# g3._gen3_overlay.capture.iq_capture_0.start()
time.sleep(captime*2)

In [ ]:
ddr4_mmio=pynq.MMIO(g3.mkidpynq.PL_DDR4_ADDR, length=2**32)

In [ ]:
ddr4_mmio.array[:]=5

In [ ]:
ddr4_mmio.array[:1024].copy()[:100]

In [ ]:
buf[:]

In [ ]:
from mkidgen3.drivers.ddc import *

In [ ]:
tone_fmt = fp_factory(*DDC.TONE_FORMAT)

In [ ]:
tone_fmt(1).__index__()

In [ ]:
los = np.linspace(f0-span/2-offset, f0+span/2-offset,500)
g3.set_lo_freq(los[0])

In [ ]:
los[0]+

In [ ]:
for lo in los:
    g3.set_lo_freq(lo)
    

## Actual Tone(s)

In [ ]:
iq_capture.register_map

In [ ]:
tones = np.array([offset*1e9])

In [ ]:
amplitudes = np.array([1.0])

In [ ]:
dactable = generateTones(frequencies=tones, n_samples=2**19, sample_rate=4.096e9, amplitudes=amplitudes)
i_data = dactable['I']
q_data = dactable['Q']
data = i_data + 1j*q_data

In [ ]:
dacreplay.stop()

In [ ]:
x=(data*2**32)
data=np.zeros_like(data, dtype=np.uint32)
data[:]|=x.real.astype(np.int16)
data[:]|=(x.imag.astype(np.int16)<<16)

In [ ]:
ol.dac_table_axim_0.replay(data)
g3._gen3_overlay.dac_table_axim_0._buffer[:10]

In [ ]:
g3._gen3_overlay.dac_table_axim_0.register_map

### Capture Configuration

In [ ]:
adcsnap.register_map.capturesize = 2**27-1 # max 2**27
adcsnap.register_map.iqout_1 = (pl_buf.device_address) & (0xffffffff) # set addresses
adcsnap.register_map.iqout_2 = (pl_buf.device_address) >> 32 # set addresses

In [ ]:
adcsnap.register_map

### Zero-PL DRAM

In [ ]:
pl_buf[:]=0

### Trigger Capture

In [ ]:
PL.ip_dict.usp_rf_data_converter_0

In [ ]:
adcsnap.register_map.CTRL.AP_START=1

In [ ]:
ps_buf = pl_buf[:2**19].copy() # first2**30 address for burst read. has to be on 4.096K boundary

In [ ]:
fp = lambda x: FpBinary(int_bits=16, frac_bits=15, signed=True, bit_field=int(x))

In [ ]:
adc_capture_data = np.zeros(ps_buf.shape[0], dtype=np.complex64)
adc_capture_data.real = [float(fp(x)) for x in ps_buf[:,0]]
adc_capture_data.imag = [float(fp(x)) for x in ps_buf[:,1]]

# Plot Captured Data

In [ ]:
N = 2**19
Fs = 4.096e9 # ADC Sample Rate [Hz]
Tc = N/Fs # total collection time (seconds)
time = np.linspace(0,Tc,N) # Time vector (seconds)
plot_start=16
plot_stop=20*4096+16
sl = slice(plot_start,plot_stop)
fft_freqs = np.linspace(-2.048e9,2.048e9,plot_stop-plot_start)

## Captured Time Series

In [ ]:
plt.figure(figsize=(10,5))
# plt.plot(time[sl],(ps_buf&0xffff)[sl].astype(np.int16), '-', label='real')
plt.plot(time[sl],adc_capture_data.real[sl])
plt.grid(True)
plt.xlabel("time(s)",position=(0.5,1))
plt.ylabel("signal(V)",position=(0,0.5))
ax = plt.gca()
ax.set_xlim(time[plot_start],time[plot_stop])
plt.legend()
plt.title('Time Series')
plt.show()

## Captured FFT

In [ ]:
y_fft = 20*np.log10(np.abs(fftshift(fft(adc_capture_data[sl]))))         # Calculate the FFT, and also shift so that f=0Hz is in the center.

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(fft_freqs[::2],y_fft[::2]) # Note: the 2./N properly normalizes the FFT amplitude to be in Volts.
plt.grid(True)
ax = plt.gca()
#plt.yscale("log")      # Uncomment these to get a log-y scale.
#ax.set_ylim(1e-2,10)
ax.set_xlim(-2.048e9,2.048e9)
plt.xlabel("Frequency[Hz]",position=(0.5,0.5))
plt.ylabel("power",position=(1,0.5))
plt.title('FFT')
plt.axvline(fft_freqs[np.argmax(y_fft)])
plt.show()

In [ ]:
fft_freqs[np.argmax(y_fft)] - -15e7

In [ ]:
n_cap = 40000000*3
adc_mb=n_cap*32/1024**2
adc_mbps=32*512
adc_eta=n_cap/512e3
print(f"Will capture {adc_mb:.0f} MB of data @ {adc_mbps} MBps. ETA {adc_eta:.0f} ms")
adc0.capture(n_cap, start=False, addr=0*2**31/2**12)

In [ ]:
print('hi')